# Processing OM10 systems with a null-deblender

In this notebook, we will discuss how we could process OM10 lensed systems with a null-deblender. 

## Requirements

OM10 and SLRealizer are the only requirements.

In [1]:
import om10
import desc.slrealizer
import warnings
warnings.filterwarnings("ignore")

/Users/Jenny/anaconda/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


After importing the packages, we have to load the OM10 catalog. We will query the systems that satisfy the criteria : `maglim=23.3,area=18000.0,IQ=0.75`.

In [2]:
db = om10.DB(catalog='/Users/Jenny/Desktop/om10fork/OM10/data/qso_mock.fits')
db.select_random(maglim=23.3,area=18000.0,IQ=0.75)
db.paint(synthetic=True)
print db.sample
realizer = desc.slrealizer.SLRealizer(catalog=db, observation="../../../data/twinkles_observation_history.csv")

OM10: Full db.lenses table contains 15658 systems
OM10: Initial db.sample contains 15658 systems
OM10: selection yields 2813 lenses
  LENSID  FLAGTYPE NIMG ZLENS ... i_SDSS_quasar z_SDSS_quasar u_SDSS_quasar
--------- -------- ---- ----- ... ------------- ------------- -------------
  2612592        0    2  0.38 ...         22.99 22.9951669685 23.3342357635
105260282        0    2 1.858 ...         22.07 22.0428427622 23.7333248961
  3342741        0    3 0.416 ...         22.41 22.2159891905 22.7401508108
  8066222        0    2 0.582 ...         21.97 21.7600933292 22.2820602793
 60363667        0    2 1.386 ...         22.43 22.3935467129 23.7408963114
  1598578        0    2 0.318 ...         20.77 20.7889812879 21.1476327903
  2926309        0    4 0.396 ...         25.11 25.1264492884 25.4975445767
 37412157        0    2 1.104 ...         21.39 21.1719496001 21.6903429195
128573065        0    2 2.086 ...         22.65  22.609311848          99.0
  8906458        0    2 0.604 ..

Here we are enabling the inline options for matplotlib and pylab so that we can see the plot in the jupyter notebook.

In [3]:
%matplotlib inline
%pylab inline
matplotlib.use('TkAgg')
matplotlib.rc('text', usetex=False)

Populating the interactive namespace from numpy and matplotlib


This process is not strictly necessary, but here we are choosing a good lens system, with large image separations and full four images.

In [4]:
wide = db.sample[np.where(db.sample['IMSEP'] > 1.0)]
print "Number of lenses with image separation > 1.0 arcsec = ",len(wide)

wide_quads = wide[np.where(wide['NIMG'] == 4)]
print "Number of these with 4 images = ",len(wide_quads)

bright_wide_quads = wide[np.where(wide_quads['MAGI'] < 22)]
print "Number of these with i3 < 22 = ",len(bright_wide_quads)

db.sample = bright_wide_quads    
                      
k = db.sample[0]['LENSID']

Number of lenses with image separation > 1.0 arcsec =  1912
Number of these with 4 images =  261
Number of these with i3 < 22 =  112


Then, we call the method `deblend` in the SLRealizer package. We need to specify the lensID and whether we want to null_deblend the objects. This method uses `skimage.measure.moments` method and plots both the original array and null-deblended array.

In [5]:
realizer.deblend(lensID=k, null_deblend=True)

OM10: Extracted OM10 lens number 1598578:
 LENSID FLAGTYPE NIMG ZLENS ... i_SDSS_quasar z_SDSS_quasar u_SDSS_quasar
------- -------- ---- ----- ... ------------- ------------- -------------
1598578        0    2 0.318 ...         20.77 20.7889812879 21.1476327903


AttributeError: 'module' object has no attribute 'abs'